In [2]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

# Read the Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/Ply.parquet"
table = pq.read_table(file_path)

# Find the index of the 'centipawn_evaluation' column
column_index = table.schema.get_field_index('centipawn_evaluation')

# Remove the 'centipawn_evaluation' column
table_without_centipawn_evaluation = table.remove_column(column_index)

# Write the updated table back to the Parquet file
pq.write_table(table_without_centipawn_evaluation, file_path)
